# TensorFlow Mechanics 101

https://www.tensorflow.org/versions/r0.10/tutorials/mnist/tf/index.html

https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/mnist/mnist.py

https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/mnist/fully_connected_feed.py

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path
import time
import math

from six.moves import xrange

import tensorflow as tf

**Load MNIST Data**

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/data')

Extracting MNIST/data/train-images-idx3-ubyte.gz
Extracting MNIST/data/train-labels-idx1-ubyte.gz
Extracting MNIST/data/t10k-images-idx3-ubyte.gz
Extracting MNIST/data/t10k-labels-idx1-ubyte.gz


In [3]:
def layer(input_tensor, units):
    input_size = input_tensor.get_shape().dims[-1].value
    weights = tf.Variable(
        tf.truncated_normal(
            [input_size, units],
            stddev=1.0 / math.sqrt(float(input_size))),
        name='weights')
    biases = tf.Variable(
        tf.zeros([units]),
        name='biases')
    return tf.matmul(input_tensor, weights) + biases

def inference(image_tensor, hidden1_units, hidden2_units, output_units):
    with tf.name_scope('hidden1'):
        hidden1 = tf.nn.relu(layer(image_tensor, hidden1_units))
    with tf.name_scope('hidden2'):
        hidden2 = tf.nn.relu(layer(hidden1, hidden2_units))
    with tf.name_scope('softmax_linear'):
        logits = layer(hidden2, output_units)
    return logits

def loss(logits, labels):
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss

def training(loss, learning_rate):
    tf.scalar_summary(loss.op.name, loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op

def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, labels, 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [4]:
def inputs(image_pixels, batch_size):
    images = tf.placeholder(
        tf.float32,
        shape=(batch_size, image_pixels))
    labels = tf.placeholder(
        tf.int32,
        shape=(batch_size))
    return images, labels

def train():
    max_steps = 2000
    batch_size = 100
    learning_rate = 0.01
    hidden1_units = 128
    hidden2_units = 32

    output_dir = 'MNIST/model'

    image_pixels = 28 * 28
    image_classes = 10

    with tf.Graph().as_default():
        X, y = inputs(image_pixels, batch_size)
        logits = inference(X, hidden1_units, hidden2_units, image_classes)
        
        loss_tensor = loss(logits, y)
        
        train_op = training(loss_tensor, learning_rate)
        
        eval_correct = evaluation(logits, y)
        
        summary_op = tf.merge_all_summaries()

        init = tf.initialize_all_variables()
        saver = tf.train.Saver()

        with tf.Session() as sess:
            summary_writer = tf.train.SummaryWriter(output_dir, sess.graph)

            sess.run(init)

            for step in xrange(max_steps):
                start_time = time.time()
            
                images, labels = mnist.train.next_batch(batch_size)
                feed_dict = {X: images, y: labels}

                _, loss_value = sess.run([train_op, loss_tensor], feed_dict=feed_dict)

                duration = time.time() - start_time
            
                if step % 100 == 0:
                    print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                    summary_str = sess.run(summary_op, feed_dict=feed_dict)
                    summary_writer.add_summary(summary_str, step)
                    summary_writer.flush()
            
                if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
                    checkpoint_file = os.path.join(output_dir, 'checkpoint')
                    saver.save(sess, checkpoint_file, global_step=step)

In [5]:
train()

Step 0: loss = 2.29 (0.039 sec)
Step 100: loss = 2.16 (0.016 sec)
Step 200: loss = 1.91 (0.007 sec)
Step 300: loss = 1.59 (0.009 sec)
Step 400: loss = 1.31 (0.025 sec)
Step 500: loss = 0.83 (0.008 sec)
Step 600: loss = 0.66 (0.007 sec)
Step 700: loss = 0.62 (0.009 sec)
Step 800: loss = 0.51 (0.009 sec)
Step 900: loss = 0.40 (0.008 sec)
Step 1000: loss = 0.53 (0.017 sec)
Step 1100: loss = 0.48 (0.355 sec)
Step 1200: loss = 0.55 (0.009 sec)
Step 1300: loss = 0.36 (0.009 sec)
Step 1400: loss = 0.43 (0.010 sec)
Step 1500: loss = 0.40 (0.008 sec)
Step 1600: loss = 0.51 (0.008 sec)
Step 1700: loss = 0.52 (0.009 sec)
Step 1800: loss = 0.35 (0.008 sec)
Step 1900: loss = 0.41 (0.011 sec)
